In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from PIL import Image, ImageDraw
from ast import literal_eval
import openslide
import cv2
import re
import json

slide_path = '/fs/ess/PAS1575/Dataset/CAMELYON16/testing/images/'
slide_mask_path = '/fs/ess/PAS1575/Dataset/CAMELYON16/testing_masks/'

output_path = './gt_and_fixation_new_zoom/'

down_scale = 2 # default value that needs to be adjusted

# get slide tissue threshold from a set of files
df_thresholds = pd.DataFrame()
for i in range(1,11):
    df_thresholds = pd.concat([pd.read_csv(f'../non_overlap_tiles/test_slides_tissue_thresholds_{i}_of_10.csv'), df_thresholds], ignore_index=True)

In [2]:
# load in eye fixation data
pids = ['P1','P3','P4','P5','P6','P7','P8','P10']

df_image = pd.read_csv('images.csv')

df_eyetrack = pd.read_csv('./TrackerData/Exp1_CombinedGazeMouseData.csv')
df_eyetrack = df_eyetrack[df_eyetrack['participantID'].isin(pids)]

eyetrack_images = df_eyetrack['imageID'].dropna().unique()
print(f"{len(eyetrack_images)} unique images in eye tracking data")

60 unique images in eye tracking data


In [3]:
# Convert to DataFrame for merging
df_test_reference = pd.read_csv('../cam16_test_reference.csv')

df_combined = pd.merge(df_image, df_test_reference, left_on='image_name', right_on='image_id')
df_combined = df_combined[~df_combined['image_id_x'].str.contains('TC')]
df_combined = df_combined[['image_id_x', 'image_name', 'type', 'level']]
df_combined.columns = ['img_id', 'image_name', 'type', 'level']

# get participantID and imageID pairs from df_eyetrack
df_eyetrack_pairs = df_eyetrack[['participantID', 'imageID']].dropna().drop_duplicates()
df_eyetrack_pairs.columns = ['pid', 'img_id']

# merge with df_combined to get levels
df_combined = pd.merge(df_combined, df_eyetrack_pairs, on='img_id')

# merge with df_combined to get correctness
df_pid_answers = pd.read_csv('../experimentData/allParticipantsAnswers.csv')
df_pid_answers = df_pid_answers[['pID', 'imageID', 'correctness']]
df_pid_answers.columns = ['pid', 'img_id', 'correctness']
df_combined = pd.merge(df_combined, df_pid_answers, on=['pid', 'img_id'])

# change none to Normal in level and delete type column
df_combined['level'] = df_combined['level'].replace('None', 'Normal')
df_combined = df_combined.drop(columns=['type'])

In [4]:
# check data
pid = 'P1'
img_id = 'C7'
df_fix = df_eyetrack[(df_eyetrack['participantID'] == pid) 
                     & (df_eyetrack['imageID'] == img_id)
                     & (df_eyetrack['EyeMovementType']=='Fixation') # Saccade, Fixation
                     # & (df_eyetrack['inMenu'] == True) # filter out very short fixations
                    ]  
df_fix = df_fix.sort_values('relativeTimestamp(ms)').copy()
df_fix = df_fix[['imageID',	'participantID', 'EyeMovementType', 'ImageFixationPointX(px)', 'ImageFixationPointY(px)', 'inMenu', 'inNavigator','zoomMagnification']]
# df_fix.tail(20)
df_fix['inNavigator'] = df_fix['inNavigator'].fillna(False)
df_fix[df_fix['inNavigator']==True]

# show rows between 15542 and 15578
df_fix[(df_fix.index >= 15592) & (df_fix.index <= 15660)]

,imageID,participantID,EyeMovementType,ImageFixationPointX(px),ImageFixationPointY(px),inMenu,inNavigator,zoomMagnification
15592,C7,P1,Fixation,20882.676281,101862.304712,False,False,1.00
15600,C7,P1,Fixation,18605.170288,93512.935473,False,False,1.00
15611,C7,P1,Fixation,44611.565878,98891.876450,False,False,2.25
15624,C7,P1,Fixation,29565.442899,96598.256963,False,False,2.25
15638,C7,P1,Fixation,27608.772198,89515.020313,False,False,2.25


In [5]:
import numpy as np
import cv2
import openslide
from PIL import Image

def _choose_level_for_downscale(slide, desired_down):
    downs = slide.level_downsamples
    idx = int(np.argmin([abs(d - desired_down) for d in downs]))
    return idx, float(downs[idx])

def _rgba_to_rgb(pil_img):
    if pil_img.mode == 'RGBA':
        return Image.alpha_composite(
            Image.new('RGBA', pil_img.size, (255, 255, 255, 255)),
            pil_img
        ).convert('RGB')
    return pil_img.convert('RGB')

def get_thumbnail_white_bg(
    slide_name,
    gt_color=(255, 0, 0),
    pad_pixels=0,
    roi_bbox_thumb=None    # (xmin_t, ymin_t, xmax_t, ymax_t) in thumbnail px
):
    """
    Reads an ROI (or full slide) using read_region, draws GT contours, 
    and preserves the original coordinate offset.

    Returns:
      thumbnail_np_out       : RGB ndarray of ROI
      thumbnail_with_gt_out  : RGB ndarray with GT contours drawn in red
      global_offset          : (xmin_global, ymin_global) — top-left coord in original slide
      bbox_used_thumb        : [xmin_t, ymin_t, xmax_t, ymax_t] in thumbnail px
    """
    # Build paths from globals
    imgPath = slide_path + slide_name + '.tif'
    imgPathMask = slide_mask_path + slide_name + '_mask.tif'

    # --- Open mask to get full dimensions and pick pyramid level ---
    with openslide.open_slide(imgPathMask) as mask_slide:
        nx, ny = mask_slide.dimensions
        L_idx, L_down = _choose_level_for_downscale(mask_slide, down_scale)

        # If no ROI, default to entire slide at thumbnail scale
        if roi_bbox_thumb is None:
            W_t = int(round(nx / down_scale))
            H_t = int(round(ny / down_scale))
            roi_bbox_thumb = (0, 0, W_t, H_t)

        xmin_t, ymin_t, xmax_t, ymax_t = map(int, roi_bbox_thumb)
        # optional pad in thumbnail units
        xmin_t = max(0, xmin_t - pad_pixels)
        ymin_t = max(0, ymin_t - pad_pixels)
        xmax_t = min(int(round(nx / down_scale)), xmax_t + pad_pixels)
        ymax_t = min(int(round(ny / down_scale)), ymax_t + pad_pixels)

        # Map to level-0 coords and compute ROI size
        xmin0 = int(round(xmin_t * down_scale))
        ymin0 = int(round(ymin_t * down_scale))
        xmax0 = int(round(xmax_t * down_scale))
        ymax0 = int(round(ymax_t * down_scale))
        w0, h0 = xmax0 - xmin0, ymax0 - ymin0

        wL = max(1, int(round(w0 / L_down)))
        hL = max(1, int(round(h0 / L_down)))

        # Read the mask ROI
        mask_rgba = mask_slide.read_region((xmin0, ymin0), L_idx, (wL, hL))
        mask_gray = np.array(_rgba_to_rgb(mask_rgba).convert('L'))

    # --- Read image ROI at same coords/level ---
    with openslide.open_slide(imgPath) as slide:
        region_rgba = slide.read_region((xmin0, ymin0), L_idx, (wL, hL))
    region_rgb = np.array(_rgba_to_rgb(region_rgba))

    # --- Draw GT contours ---
    gt_mask = (mask_gray > 0).astype(np.uint8)
    contours, _ = cv2.findContours(gt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    region_rgb_with_gt = region_rgb.copy()
    if contours:
        roi_bgr = cv2.cvtColor(region_rgb_with_gt, cv2.COLOR_RGB2BGR)
        cv2.drawContours(roi_bgr, contours, -1, gt_color, int(1*32/down_scale))
        region_rgb_with_gt = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2RGB)

    # Return image, overlay, and the global offset (top-left in original coords)
    global_offset = (xmin0, ymin0)
    bbox_used_thumb = [xmin_t, ymin_t, xmax_t, ymax_t]

    return region_rgb, region_rgb_with_gt, global_offset, bbox_used_thumb

In [6]:
def get_fixations_for_slide(pid, img_id, df_eyetrack, down_scale=32):
    """Filter and clean fixations for a given participant and image, then return scaled X, Y and sizes.

    This version computes df_fix internally and returns only (X, Y, sizes).
    Coordinates are scaled by `down_scale` to match thumbnails.
    """
    # Select relevant fixations and sort
    df_fix = df_eyetrack[(df_eyetrack['participantID'] == pid) &
                         (df_eyetrack['imageID'] == img_id) &
                         (df_eyetrack['EyeMovementType']=='Fixation')]
    
    # remove data whose X are nan
    df_fix = df_fix[pd.notna(df_fix['ImageFixationPointX(px)']) & pd.notna(df_fix['ImageFixationPointY(px)'])]

    # sort by time
    df_fix = df_fix.sort_values('relativeTimestamp(ms)').copy()

    # Ensure column exists and coerce to numeric
    df_fix['GazeEventDuration(ms)'] = pd.to_numeric(df_fix.get('GazeEventDuration(ms)', pd.Series(dtype=float)), errors='coerce')

    # Replace remaining NaN or negative durations with zero
    df_fix['GazeEventDuration(ms)'] = df_fix['GazeEventDuration(ms)'].fillna(0).clip(lower=0)

    # Compute scaled coordinates and sizes
    X = (df_fix['ImageFixationPointX(px)'] / down_scale).astype(float).tolist()
    Y = (df_fix['ImageFixationPointY(px)'] / down_scale).astype(float).tolist()
    sizes = df_fix['GazeEventDuration(ms)'].astype(float).tolist()
    zooms = df_fix['zoomMagnification'].astype(float).tolist()

    return X, Y, sizes, zooms

In [7]:
# Menu jump detection and marker definitions
XCOL = 'ImageFixationPointX(px)'
YCOL = 'ImageFixationPointY(px)'
IMCOL = 'inMenu'

def get_menu_jump_indices(df, require_return_fixation=False):
    """
    Mark index i if:
      - row i is a valid fixation (not in menu, finite XY)
      - and there is at least one inMenu=True row between row i and the next valid fixation.
        If require_return_fixation=True, only mark when a *next* fixation exists.
        If False, also mark the last fixation if the sequence ends with a menu run.
    Returns a list of integer row indices (relative to df) to annotate.
    """
    d = df.copy()

    # 1) Coerce inMenu to pure booleans
    if IMCOL not in d.columns:
        d[IMCOL] = False
    d[IMCOL] = d[IMCOL].fillna(False).astype(bool)

    # 2) Valid fixation rows (not menu + finite XY)
    x = d[XCOL].to_numpy()
    y = d[YCOL].to_numpy()
    in_menu = d[IMCOL].to_numpy()

    finite_xy = np.isfinite(x) & np.isfinite(y)
    valid = (~in_menu) & finite_xy
    valid_idx = np.flatnonzero(valid)

    if valid_idx.size == 0:
        return []

    marks = []
    n = len(d)

    # 3) For each fixation i, scan the slice until the next fixation
    for k, i in enumerate(valid_idx):
        j_next = valid_idx[k+1] if (k+1) < valid_idx.size else n  # end if last fixation
        if i+1 >= j_next:  # no rows between i and next fixation
            continue

        # any menu rows in (i, j_next)?
        if in_menu[i+1:j_next].any():
            # Optionally require a return fixation
            if require_return_fixation and j_next == n:
                # sequence ended after a menu run — skip marking if you don't want trailing-menu marks
                continue
            marks.append(int(i))

    return marks


from matplotlib.path import Path
import matplotlib.markers as mmarkers

def make_hamburger_marker():
    """
    Create a hamburger (menu) marker: three horizontal bars as a single Path.
    Coordinates are in roughly [-0.5, 0.5] box so 's' in scatter scales nicely.
    """
    bars = [
        (-0.5,  0.35), (0.5,  0.35), (0.5,  0.50), (-0.5,  0.50), (-0.5,  0.35),  # top bar
        (-0.5, -0.075), (0.5, -0.075), (0.5,  0.075), (-0.5,  0.075), (-0.5, -0.075),  # middle bar
        (-0.5, -0.50), (0.5, -0.50), (0.5, -0.35), (-0.5, -0.35), (-0.5, -0.50)   # bottom bar
    ]
    # Build codes: MOVETO + 3 LINETOs + CLOSEPOLY for each rectangle (5 verts per bar)
    codes = []
    verts = []
    for bi in range(3):
        i0 = bi * 5
        verts.extend(bars[i0:i0+5])
        codes.extend([Path.MOVETO, Path.LINETO, Path.LINETO, Path.LINETO, Path.CLOSEPOLY])
    path = Path(verts, codes)
    return mmarkers.MarkerStyle(path)

In [8]:
# Navigation jump detection
def get_navigator_jump_indices(df):
    """Return fixation row indices followed by inNavigator=True rows."""
    d = df.copy()
    if 'inNavigator' not in d.columns:
        return []
    d['inNavigator'] = d['inNavigator'].fillna(False).astype(bool)

    xcol, ycol = 'ImageFixationPointX(px)', 'ImageFixationPointY(px)'
    x = d[xcol].to_numpy()
    y = d[ycol].to_numpy()
    nav = d['inNavigator'].to_numpy()
    finite_xy = np.isfinite(x) & np.isfinite(y)
    valid = (~nav) & finite_xy
    valid_idx = np.flatnonzero(valid)

    marks = []
    n = len(d)
    for k, i in enumerate(valid_idx):
        j_next = valid_idx[k+1] if (k+1) < valid_idx.size else n
        if i+1 >= j_next:
            continue
        if nav[i+1:j_next].any():
            marks.append(int(i))
    return marks

In [9]:
# set participant and image
pid = 'P1'
img_id = 'C1'  # You can change this to test different images

In [10]:
# Test tissue detection
slide_name = df_image.at[df_image.index[df_image['image_id']==img_id].values[0], 'image_name']
print(f"Processing slide: {slide_name}")

# Example: crop a  thumb-ROI (xmin,ymin,xmax,ymax) in THUMBNAIL pixels
xmin, ymin, xmax, ymax = 450*32/down_scale, 320*32/down_scale, 830*32/down_scale, 620*32/down_scale
roi_thumb = (xmin, ymin, xmax, ymax)
img_roi, img_roi_with_gt, offset, bbox_used = get_thumbnail_white_bg(
    slide_name=slide_name,
    gt_color=(0,0,255),
    pad_pixels=0,
    roi_bbox_thumb=roi_thumb
)

# fig, ax = plt.subplots(figsize=(16, 16), dpi=300)
# ax.imshow(
#     img_roi_with_gt,
#     extent=[xmin, xmax, ymax, ymin],  # [xmin, xmax, ymax, ymin]
#     origin='upper', interpolation='none'
# )

# # Add legend for ground truth
# from matplotlib.patches import Patch
# legend_elements = [
#     Patch(facecolor='none', edgecolor='red', label='Ground Truth')
# ]
# ax.legend(handles=legend_elements, loc='upper left')

Processing slide: test_001


In [37]:
def add_zoom_scale(ax, x0, y0, down_scale, ticks=(1, 10, 20, 40),
                   color='blue', bar_lw=8, tick_len=15, tick_lw=2,
                   label='Zoom', text_size=10):
    # Same mapping as your bars: 1×→10, 40×→200, then * 32/down_scale
    def H(m):  # height in data units for magnification m
        return (10 + (m - 1) * (200 - 10) / (40 - 1)) * 32 / down_scale

    Hmax = H(max(ticks))

    # Main vertical bar
    ax.plot([x0, x0], [y0 - Hmax, y0], color=color, lw=bar_lw, alpha=0.8, zorder=100)

    # Tick marks + labels
    for t in ticks:
        yt = y0 - H(t)
        ax.plot([x0 - tick_len + 1, x0 + tick_len - 1], [yt, yt], color='white', lw=tick_lw, zorder=101)
        ax.text(x0 + tick_len + 30, yt, f'{t}×', va='center', ha='left', fontsize=text_size, zorder=102)

    # Title under/above bar (adjust for your coordinate origin)
    ax.text(x0+180, y0 - Hmax - 230, label, va='top', ha='center', fontsize=text_size+2, zorder=102)


In [39]:
# Filter to this participant+image
df_eye = df_eyetrack[(df_eyetrack["imageID"] == img_id)
                     & (df_eyetrack["participantID"] == pid)].copy()
df_eye = df_eye.sort_values('relativeTimestamp(ms)').copy()
df_eye['inMenu'] = df_eye['inMenu'].fillna(False).astype(bool)

# Prepare plotting
fig, axs = plt.subplots(figsize=(16, 16), dpi=300)
axs.imshow(
    img_roi_with_gt,
    extent=[xmin, xmax, ymax, ymin],  # [xmin, xmax, ymax, ymin]
    origin='upper', interpolation='none'
)

# Get scaled fixation coordinates and sizes using helper (df_fix is handled inside the helper)
X, Y, size, zoom = get_fixations_for_slide(pid, img_id, df_eye, down_scale=down_scale)

# Create arrows between consecutive points
# Calculate the differences between consecutive points for arrow directions
dx = np.diff(X)
dy = np.diff(Y)
# Starting points of arrows (all points except the last)
X_start = X[:-1]
Y_start = Y[:-1]
# Plot arrows with a medium gray color
axs.quiver(X_start, Y_start, dx, dy, scale_units='xy', angles='xy', scale=1, 
          color='#404040', width=0.005, headwidth=4, headlength=5, headaxislength=4.5)

# Prepare sizes and colors for scatter
sizes_np = np.array(size, dtype=float)
# default to small positive if empty
if sizes_np.size == 0:
    sizes_np = np.array([1.0])

# scale sizes for plotting (tweak divisor/min for visibility)
sizes_plot = np.clip(sizes_np, 10, 1000)

# Create fixed duration bins (0-200, 200-400, ..., 800-1000)
bins = np.array([0, 200, 400, 600, 800, 1000])
n_bins = len(bins) - 1
bin_indices = np.clip(np.digitize(sizes_np, bins) - 1, 0, n_bins-1)  # Clip to handle durations > 1000ms

# Use colorblind-safe and print friendly colors from Color Brewer
colors = ["#FC7405", "#eae48d", '#78c679', "#43a4ca", "#7585FC"] # choice 14

discrete_cmap = mpl.colors.ListedColormap(colors[:n_bins])

sc = axs.scatter(X, Y, s=sizes_plot, c=bin_indices, 
                cmap=discrete_cmap, alpha=1, 
                edgecolors='white', linewidths=0.5, zorder=3)

# Add colorbar showing gaze duration values
cbar = plt.colorbar(sc, ax=axs, fraction=0.03, pad=0.02,
                    shrink=0.46,
                   boundaries=np.arange(n_bins + 1),
                   values=np.arange(n_bins))
# Label with all duration values including 1000
cbar.set_ticks(np.arange(n_bins + 1))
duration_labels = [str(int(val)) for val in bins]
cbar.ax.set_yticklabels(duration_labels, fontsize=20)
cbar.set_label('Fixation time (ms)', fontsize=22, labelpad=10)


### add zoom bars above fixation points

# Compute normalized zoom (0 → 1)
zoom_np = np.array(zoom, dtype=float)
zoom_np = np.clip(np.nan_to_num(zoom_np, nan=1.0), 1, 40)
# Linear mapping: 1× → 10, 40× → 200
bar_heights = (10 + (zoom_np - 1) * (200 - 10) / (40 - 1)) * 32 / down_scale

# Draw a thin vertical line above each fixation
for xi, yi, bh in zip(X, Y, bar_heights):
    axs.plot([xi-3, xi-3], [yi - bh, yi],   # slightly above the dot
             color='blue', lw=6, zorder=12, alpha=0.8)

# If your image uses origin='upper' (y increases downward):
x0 = xmin + 100   # right margin
y0 = ymax - 1300   # top margin

# add zoom scale bar
add_zoom_scale(axs, x0, y0, down_scale, text_size=20)


# set x and y limits
axs.set_xlim(xmin, xmax)
axs.set_ylim(ymax, ymin)

# Add axis labels
axs.set_xlabel('X (pixels/2)', fontsize=22)
axs.set_ylabel('Y (pixels/2)', fontsize=22)

# set ticklabel size
axs.tick_params(axis='both', which='major', labelsize=20)

# Add legend
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor='none', edgecolor='red', lw=1.5 * 1.5, label='Ground truth'),
    Line2D([0], [0], color='#404040', lw=2 * 1.5, label='Saccade'),
    Line2D([0], [0], marker='o', color='w', label='Fixations',
           markerfacecolor='#43a4ca', markersize=10 * 1.5, markeredgecolor='white'),
    Line2D([0], [0], marker='|', linestyle='None',
           markeredgecolor='blue', markerfacecolor='none',
           markersize=15 * 1.5, markeredgewidth=3.5 * 1.5, label='Zoom level')
]

axs.legend(
    handles=legend_elements,
    loc='best',
    frameon=True,
    fontsize=19,         # text size
    handlelength=1.9 * 1.5,    # line handle length
    handleheight=1.3 * 1.5,    # vertical handle height
    borderpad=0.6 * 1.5,       # padding inside legend box
    labelspacing=0.5 * 1.5     # spacing between entries
)
# axs.axis('off')

# save figure
output_filename = os.path.join(output_path, f'teaser_{pid}_{img_id}_{slide_name}_clean.png')
plt.savefig(output_filename, bbox_inches='tight')
print(f"Figure saved to: {output_filename}")
plt.close(fig)

Figure saved to: ./gt_and_fixation_new_zoom/teaser_P1_C1_test_001_clean.png


### Complicated Version

In [ ]:
# Filter to this participant+image
df_eye = df_eyetrack[(df_eyetrack["imageID"] == img_id)
                     & (df_eyetrack["participantID"] == pid)].copy()
df_eye = df_eye.sort_values('relativeTimestamp(ms)').copy()
df_eye['inMenu'] = df_eye['inMenu'].fillna(False).astype(bool)

# Compute menu-jump anchors (indices in df_eye)
menu_mark_indices = get_menu_jump_indices(df_eye)
print(f"Menu jump anchor fixations at rows: {menu_mark_indices}")

# Compute navigator jump anchors
nav_mark_indices = get_navigator_jump_indices(df_eye)
print(f"Navigator jump anchor fixations at rows: {nav_mark_indices}")

# Prepare plotting
fig, axs = plt.subplots(figsize=(16, 16), dpi=300)
axs.imshow(
    img_roi_with_gt,
    extent=[xmin, xmax, ymax, ymin],  # [xmin, xmax, ymax, ymin]
    origin='upper', interpolation='none'
)

# Get scaled fixation coordinates and sizes using helper (df_fix is handled inside the helper)
X, Y, size, zoom = get_fixations_for_slide(pid, img_id, df_eye, down_scale=down_scale)

# Create arrows between consecutive points
# Calculate the differences between consecutive points for arrow directions
dx = np.diff(X)
dy = np.diff(Y)
# Starting points of arrows (all points except the last)
X_start = X[:-1]
Y_start = Y[:-1]
# Plot arrows with a medium gray color
axs.quiver(X_start, Y_start, dx, dy, scale_units='xy', angles='xy', scale=1, 
          color='#404040', width=0.0025, headwidth=4, headlength=5, headaxislength=4.5)

# Prepare sizes and colors for scatter
sizes_np = np.array(size, dtype=float)
# default to small positive if empty
if sizes_np.size == 0:
    sizes_np = np.array([1.0])

# scale sizes for plotting (tweak divisor/min for visibility)
sizes_plot = np.clip(sizes_np, 10, 1000)

# Create fixed duration bins (0-200, 200-400, ..., 800-1000)
bins = np.array([0, 200, 400, 600, 800, 1000])
n_bins = len(bins) - 1
bin_indices = np.clip(np.digitize(sizes_np, bins) - 1, 0, n_bins-1)  # Clip to handle durations > 1000ms

# Use colorblind-safe and print friendly colors from Color Brewer
colors = ["#FC7405", "#eae48d", '#78c679', "#43a4ca", "#7585FC"] # choice 14

discrete_cmap = mpl.colors.ListedColormap(colors[:n_bins])

sc = axs.scatter(X, Y, s=sizes_plot, c=bin_indices, 
                cmap=discrete_cmap, alpha=1, 
                edgecolors='white', linewidths=0.5, zorder=3)

# Add colorbar showing gaze duration values
cbar = plt.colorbar(sc, ax=axs, fraction=0.03, pad=0.04,
                    shrink=0.5,
                   boundaries=np.arange(n_bins + 1),
                   values=np.arange(n_bins))
# Label with all duration values including 1000
cbar.set_ticks(np.arange(n_bins + 1))
duration_labels = [str(int(val)) for val in bins]
cbar.ax.set_yticklabels(duration_labels)
cbar.set_label('Fixation time (ms)')

### add zoom bars above fixation points
# Compute normalized zoom (0 → 1)
zoom_np = np.array(zoom, dtype=float)
zoom_np = np.clip(np.nan_to_num(zoom_np, nan=1.0), 1, 40)
# Linear mapping: 1× → 10, 40× → 400
bar_heights = (10 + (zoom_np - 1) * (400 - 10) / (40 - 1)) * 32 / down_scale

# Draw a thin vertical line above each fixation
for xi, yi, bh in zip(X, Y, bar_heights):
    axs.plot([xi-5, xi-5], [yi - bh, yi],   # slightly above the dot
             color='blue', lw=5, zorder=12, alpha=0.8)
    

# Use a clean black diamond or square
navigator_marker = 'D'   # diamond; you can switch to 's' (square) or '^' (triangle)
for ridx in nav_mark_indices:
    fx = df_eye.iloc[ridx][XCOL] / down_scale
    fy = df_eye.iloc[ridx][YCOL] / down_scale
    axs.scatter([fx], [fy],
                s=500, marker=navigator_marker,
                facecolor='none', edgecolor='blue', linewidth=1.5,
                zorder=9,
                label='Navigator jump' if ridx == nav_mark_indices[0] else "")


# Menu-jump hamburger markers at the *anchor fixation* positions
hamburger_marker = make_hamburger_marker()
for ridx in menu_mark_indices:                    # ridx is positional
    fx = df_eye.iloc[ridx][XCOL] / down_scale
    fy = df_eye.iloc[ridx][YCOL] / down_scale
    axs.scatter([fx], [fy],
                s=600, marker=hamburger_marker,   # bigger so it’s visible
                facecolor="#2D2D2DFF", edgecolor='black', linewidth=1,
                zorder=10, label='Menu jump' if ridx == menu_mark_indices[0] else "")

# set x and y limits
axs.set_xlim(xmin, xmax)
axs.set_ylim(ymax, ymin)

# Add axis labels
axs.set_xlabel('X (pixels)')
axs.set_ylabel('Y (pixels)')

# Add legend
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor='none', edgecolor='red', lw=1.5 * 1.2, label='Ground Truth'),
    Line2D([0], [0], color='#404040', lw=2 * 1.2, label='Saccade'),
    Line2D([0], [0], marker='o', color='w', label='Fixations',
           markerfacecolor='#43a4ca', markersize=9 * 1.2, markeredgecolor='white'),
    Line2D([0], [0], marker='D', color='w', label='Navigator jump',
           markerfacecolor='none', markeredgecolor='blue', markersize=8 * 1.2, lw=2 * 1.2),
    Line2D([0], [0], marker=make_hamburger_marker(), color='w', label='Menu jump',
           markerfacecolor='#2D2D2DFF', markeredgecolor='black', markersize=10 * 1.2),
    Line2D([0], [0], marker='|', linestyle='None',
           markeredgecolor='blue', markerfacecolor='none',
           markersize=14 * 1.2, markeredgewidth=2 * 1.2, label='Zoom level')
]

axs.legend(
    handles=legend_elements,
    loc='best',
    frameon=True,
    fontsize=10 * 1.2,         # text size
    handlelength=2.0 * 1.2,    # line handle length
    handleheight=1.2 * 1.2,    # vertical handle height
    borderpad=0.8 * 1.2,       # padding inside legend box
    labelspacing=0.6 * 1.2     # spacing between entries
)
# axs.axis('off')

# save figure
output_filename = os.path.join(output_path, f'teaser_{pid}_{img_id}_{slide_name}.png')
plt.savefig(output_filename, bbox_inches='tight')
print(f"Figure saved to: {output_filename}")